<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">سنجش میزان رضایت از ضمانت کالا</h1>


<h2 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:15px; margin-bottom:10px;">وارد کردن کتابخانه و لود دیتا</h2>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
در قدم نخست، کتابخانه‌های کلیدی شامل پانداس برای دستکاری داده‌ها و ابزارهای پردازش متن فراخوانی می‌شوند.
همچنین داده های آموزش و نگاشت از روی درایو خوانده میشوند.
</p>


In [2]:
import pandas as pd
import gdown
import re
!pip install gensim
import gensim.downloader as api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 55.5 MB/s eta 0:00:00


In [3]:
gdown.download("https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW", "train_data.csv", quiet =True)

'train_data.csv'

In [9]:
gdown.download("https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h", "mapping.csv", quiet=True)

'mapping.csv'

In [4]:
df_train =pd.read_csv("train_data.csv", low_memory =False)
print(df_train.shape)

(838944, 11)


<h2 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:15px; margin-bottom:10px;">استانداردسازی داده های نگاشت</h2>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
نام فروشندگان غالبا با پیشوندها و پسوندهای مربوط به لینک‌های رابط کاربری آمازون ترکیب شده است. برای حفظ یکپارچگی داده‌ها در عملیات ادغام پیش رو، از عبارات باقاعده استفاده کردیم تا با تشخیص دقیق مرزهای ابتدا و انتهای رشته، این نویزهای اضافی به طور کامل و بدون آسیب به نام اصلی برندها پاکسازی شوند.
</p>


In [11]:
dm =pd.read_csv("mapping.csv", low_memory =False)

dm['brand'] =dm['brand'].astype(str).str.replace(r"^Visit Amazon's\s+", "", regex =True)
dm['brand'] =dm['brand'].str.replace(r"\s+Page$", "", regex =True)
dm['brand'] =dm['brand'].str.replace(r"\s+Store$", "", regex =True)

<h2 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:15px; margin-bottom:10px;">استفاده از مدل fast text برای پیدا کردن کلمات مشابه</h2>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
مدل‌های تعبیه کلمه کلاسیک مانند word2vec در مواجهه با کلمات دارای غلط املایی دچار خطای کلمات خارج از دایره لغات می‌شوند. برای حل ریشه‌ای این چالش، مدل فست تکست با فضای برداری ۳۰۰ بعدی را به پروژه اضافه نمودیم. این معماری به جای نگاه یکپارچه به کلمات، آن‌ها را به زیرکلمه‌ها می‌شکند. این ویژگی استثنایی اجازه می‌دهد تا علاوه بر یافتن مترادف‌های دقیق، تمامی اشتباهات املایی رایج کاربران برای کلمات گارانتی و ضمانت نیز با بالاترین دقت ممکن استخراج شوند. در این مرحله، شبکه‌ای از نزدیک‌ترین کلمات در فضای برداری شناسایی و پس از تجمیع، برای بررسی نهایی آماده می‌شوند.
</p>


In [5]:
m =api.load("fasttext-wiki-news-subwords-300")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [6]:
w1 =m.most_similar("warranty", topn =50)
w2 =m.most_similar("guarantee", topn =50)

In [7]:
l1 =[x[0] for x in w1]
l2 =[x[0] for x in w2]
w =list(set(l1 + l2 + ["warranty", "guarantee"]))
w.sort()
print("Total unique similar words:")
print(len(w))
print("List of words:")
print(w)

Total unique similar words:
94
List of words:
['AppleCare', 'Warranties', 'Warranty', 'adequate', 'after-sale', 'after-sales', 'anty', 'assurance', 'assurances', 'assure', 'assured', 'assures', 'assuring', 'backorder', 'chanty', 'collateralize', 'constitutionally-guaranteed', 'contract', 'contractual', 'court-enforceable', 'deposit-guarantee', 'drive-away', 'due-on-sale', 'durability', 'ensurance', 'ensure', 'ensured', 'ensures', 'ensuring', 'full-proof', 'garantee', 'gaurantee', 'government-guaranteed', 'guaranted', 'guarantee', 'guaranteed', 'guaranteeing', 'guarantees', 'guarantied', 'guaranties', 'guarantor', 'guarantors', 'guaranty', 'guarantying', 'guarentee', 'guarrantee', 'guarranteed', 'in-force', 'insurability', 'insure', 'loan-guarantee', 'maintenance', 'merchantability', 'model-year', 'money-back', 'new-vehicle', 'out-of-warranty', 'planty', 'post-sale', 'powertrain', 'pre-commit', 'product', 'promise', 'promised', 'promises', 'promiss', 'protection', 'provide', 'ranty', 'r

<h2 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:15px; margin-bottom:10px;">پالایش واژگان، فیلترینگ متنی و رتبه‌بندی رضایت کاربران</h2>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
خروجی خام مدل‌های زبانی نیازمند نظارت و پالایش انسانی است تا از ورود کلمات عمومی که منجر به کاهش دقت سیستم می‌شوند جلوگیری گردد. پس از تشکیل یک لیست جامع و تمیز از واژگان هدف و خطاهای تایپی آن‌ها، یک الگوی جستجوی متنی پیشرفته با رعایت مرز دقیق کلمات طراحی شد. با اعمال این الگو روی بدنه اصلی نظرات، تنها بازخوردهای مرتبط با خدمات پس از فروش غربال شدند.
در نهایت میانگین امتیاز و تعداد نظرات برای هر کالا محاسبه شده و سپس این نتایج با جدول نگاشت ادغام شدند. خروجی نهایی، جدولی ساختاریافته از محصولاتی است که  بالاترین میزان رضایت از گارانتی را ثبت کرده‌اند.
</p>


In [12]:
c_words =['applecare', 'warranties', 'warranty', 'warrantee', 'warrantees', 'warrantied', 'warrantor', 'warrenty', 'out-of-warranty', 'guarantee', 'guaranteed', 'guarantees', 'guaranteeing', 'guaranty', 'guaranties', 'guarantied', 'guarantor', 'guarantors', 'guarantying', 'garantee', 'gaurantee', 'guaranted', 'guarentee', 'guarrantee', 'guarranteed', 'assurance', 'assurances', 'money-back', 'repair', 'maintenance', 'protection', 'after-sale', 'after-sales', 'durability']
p_reg =r"\b(" + "|".join(c_words) + r")\b"

In [13]:
df_train['reviewText'] =df_train['reviewText'].fillna("")
m_war =df_train['reviewText'].str.contains(p_reg, flags =re.IGNORECASE, regex =True)
df_filt =df_train[m_war]
g_mean =df_filt.groupby('asin')['overall'].mean().reset_index(name ='war_mean')
g_cnt =df_filt.groupby('asin').size().reset_index(name ='war_count')

/tmp/ipython-input-1269719304.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_war =df_train['reviewText'].str.contains(p_reg, flags =re.IGNORECASE, regex =True)


In [14]:
res_war =pd.merge(g_mean, g_cnt, on ='asin')
res_fin =pd.merge(res_war, dm[['asin', 'title', 'brand']], on ='asin', how ='left')
res_fin =res_fin.sort_values('war_count', ascending =False).reset_index(drop =True)
top_20 =res_fin[['brand', 'title', 'war_mean', 'war_count']].head(20)

In [19]:
from IPython.display import display, HTML
display(HTML("<h3>--- Product Warranty Satisfaction (Top 20) ---</h3>"))
display(top_20)

,brand,title,war_mean,war_count
0,AmazonBasics,AmazonBasics 14-Inch Laptop Sleeve - Black,4.513043,115
1,Belkin,Belkin 8-Outlet Power Strip Surge Protector wi...,4.280899,89
2,APC,APC 11-Outlet Surge Protector 3020 Joules with...,4.533333,75
3,Asus,ASUS Tri-Band Gigabit (AC3200) WiFi Router (Up...,3.031746,63
4,Microsoft,Microsoft Band 2 - Medium,1.890909,55
5,tomtoc,tomtoc 360&deg; Protective Laptop Sleeve Compa...,4.592593,54
6,Samsung,Samsung 850 PRO - 1TB - 2.5-Inch SATA III Inte...,4.056604,53
7,Samsung,Samsung EVO 64GB 48/MB/s Micro SDXC Memory Ca...,1.725490,51
8,Cambridge Soundworks,OontZ Angle 3 Enhanced Stereo Edition IPX5 Spl...,3.760000,50
9,Western Digital,WD Black 750GB Performance Mobile Hard Disk Dr...,4.220000,50
